In [1]:
#@title Install required libraries
!pip install -q openai

In [48]:
#@title Import libraries and setup
import json
from openai import OpenAI
from google.colab import userdata

# Initialize OpenAI client with secure API key
client = OpenAI(
    api_key=userdata.get("OPENAI_API_KEY")  # From Colab secrets
)

In [3]:
#@title Data Loading Functions
def load_kg_data(file_path):
    """Load Knowledge Graph JSON file as raw string"""
    try:
        with open(file_path, 'r') as f:
            return json.dumps(json.load(f)), None
    except Exception as e:
        return None, f"Error loading KG file: {str(e)}"

def load_raw_data(file_path):
    """Load raw text file as string"""
    try:
        with open(file_path, 'r') as f:
            return f.read(), None
    except Exception as e:
        return None, f"Error loading raw file: {str(e)}"

In [49]:
#@title Prompt Templates
KG_PROMPT = """
1. Focus on the question:
Analyze the provided TraceCompass State System knowledge graph data to answer the question. You must focus on answering this specific question, and your response should be derived exclusively from the provided knowledge graph data.

2. Use the provided data:
The data you will use is from the TraceCompass State System, specifically in the form of a knowledge graph. Ensure your analysis strictly uses this data for the answer.

3. Leverage your knowledge:
You may use your understanding of TraceCompass State System analysis methods (such as CPU usage analysis, active thread analysis, and Ease script queries) as context for interpreting the data. Keep in mind that the provided data comes from an Ease script query run on the TraceCompass system.

4. Reason over the graph:
The data may not always be explicitly present in the graph, and you may need to reason over the graph's structure and relationships to derive the answer. Use your knowledge of node relationships, temporal patterns, and resource utilization to infer the missing information where necessary.

5. Answer directly:
Provide a direct, concise answer to the question. This answer should be based solely on the provided data.

6. Explain your reasoning:
After providing the direct answer, explain how you arrived at it. Your explanation should cover the following:
   - How the nodes and their relationships helped answer the question.
   - Any specific behaviors or insights derived from the TraceCompass State System analysis, such as CPU usage or thread activity analysis.

7. Question and context:
Question: {question}
TraceCompass Knowledge Graph Context:
{context}

"""

RAW_PROMPT = """
1. Focus on the question:
Analyze the provided raw TraceCompass system trace data to answer the question. Your response must be directly based on this data.

2. Use the provided data:
The data you will use is from the TraceCompass State System in the form of raw system trace measurements. Your analysis should rely exclusively on this data for your answer.

3. Leverage your knowledge:
You may use your understanding of TraceCompass State System analysis methods (such as CPU usage analysis, active thread analysis, and Ease script queries) to interpret the trace data. The provided data is the result of an Ease script query run on the TraceCompass system.

4. Answer directly:
Provide a direct, concise answer to the question based on the raw trace data.

5. Explain your reasoning:
After providing the direct answer, explain how the raw data supports your answer. In your explanation, you should cover:
   - Any patterns or anomalies you identified in the trace data.
   - How these patterns or anomalies are relevant to the TraceCompass system's performance or behavior, particularly related to CPU usage, thread activity, or any Ease script queries.
   - Any performance characteristics or insights drawn from the data, emphasizing the specific context of TraceCompass' analysis.

6. Question and context:
Question: {question}
Raw Data Context:
{context}


"""

In [52]:
#@title Analysis Functions
def query_gpt4(prompt):
    """Execute GPT-4o query using modern client syntax"""
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,
            # reasoning_effort="medium",  # Options: "low", "medium", "high"
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"API Error: {str(e)}"

def analyze_kg(question, file_path):
    """Knowledge Graph analysis pipeline"""
    context, error = load_kg_data(file_path)
    if error:
        return error

    prompt = KG_PROMPT.format(
        question=question,
        context=context[:15000]  # Context window management
    )
    return query_gpt4(prompt)

def analyze_raw(question, file_path):
    """Raw data analysis pipeline"""
    context, error = load_raw_data(file_path)
    if error:
        return error

    prompt = RAW_PROMPT.format(
        question=question,
        context=context[:15000]  # Keep within token limits
    )
    return query_gpt4(prompt)

In [57]:
#@title Main Execution
def full_analysis(question, kg_path="data/cpu_usage_graph.json", raw_path="data/cpu_usage_input.txt"):
    """Run complete analysis with both approaches"""
    print(f"\n🔍 Question: {question}")
    print("="*60)

    print("\n\n📈 Raw Data Analysis:")
    print(analyze_raw(question, raw_path))

    print("\n📊 Knowledge Graph Analysis:")
    print(analyze_kg(question, kg_path))



# Example usage
if __name__ == "__main__":
    sample_question = "What is the total accumulated CPU time for thread 5130 on CPU 2?"  #@param {type:"string"}
    full_analysis(sample_question)


🔍 Question: What is the total accumulated CPU time for thread 5130 on CPU 2?


📈 Raw Data Analysis:
The total accumulated CPU time for thread 5130 on CPU 2 is **12,974,809**.

**Explanation:**

To determine the total accumulated CPU time for thread 5130 on CPU 2, we examine the raw data provided. The data is structured in a format where each entry follows the pattern `CPUs/<CPU number>/<Thread ID> => <CPU time>`. We need to locate the entry that corresponds to CPU 2 and thread 5130:

- **CPUs/2/5130 => 12974809**

This entry indicates that thread 5130 on CPU 2 has an accumulated CPU time of 12,974,809. There are no other entries for thread 5130 on CPU 2 in the provided data, confirming that this value is the total accumulated CPU time for that specific thread on that specific CPU.

📊 Knowledge Graph Analysis:
The total accumulated CPU time for thread 5130 on CPU 2 is 12,974,809 nanoseconds.

**Explanation:**

1. **Node Identification:**
   - The graph provides nodes representing both 